In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt

2024-02-03 05:21:19.908335: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Veriyi indirme ve işleme adımları
ticker_symbol = "AAPL"
start_date = "2010-01-01"
end_date = "2022-01-01"

data = yf.download(ticker_symbol, start=start_date, end=end_date)
close_prices = data['Close'].values.reshape(-1, 1)

scaler = MinMaxScaler()
close_prices_scaled = scaler.fit_transform(close_prices)

[*********************100%%**********************]  1 of 1 completed


In [3]:
# Veriyi eğitim ve test setlerine ayırma
train_size = int(len(close_prices_scaled) * 0.80)
train_data, test_data = close_prices_scaled[:train_size], close_prices_scaled[train_size:]

# Zaman serisi verilerini oluşturma fonksiyonu
def create_time_series_data(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)


In [4]:

look_back = 60  # Zaman serisi uzunluğu
X_train, y_train = create_time_series_data(train_data, look_back)
X_test, y_test = create_time_series_data(test_data, look_back)

# Verileri LSTM modeline uygun hale getirme
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


In [5]:
# Gelişmiş LSTM modeli oluşturma
model = Sequential()
model.add(LSTM(units=100, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=100))
model.add(Dropout(0.2))
model.add(Dense(units=1))

model.compile(loss='mean_squared_error', optimizer='adam')

In [6]:
# Modeli eğitme
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
74/74 [==============================] - 8s 46ms/step - loss: 6.5320e-04
Epoch 2/10
74/74 [==============================] - 3s 43ms/step - loss: 1.3033e-04
Epoch 3/10
74/74 [==============================] - 3s 44ms/step - loss: 1.0933e-04
Epoch 4/10
74/74 [==============================] - 3s 46ms/step - loss: 1.1270e-04
Epoch 5/10
74/74 [==============================] - 3s 45ms/step - loss: 8.8761e-05
Epoch 6/10
74/74 [==============================] - 3s 46ms/step - loss: 1.0121e-04
Epoch 7/10
74/74 [==============================] - 3s 46ms/step - loss: 8.0444e-05
Epoch 8/10
74/74 [==============================] - 3s 45ms/step - loss: 9.1752e-05
Epoch 9/10
74/74 [==============================] - 3s 46ms/step - loss: 8.1394e-05
Epoch 10/10
74/74 [==============================] - 3s 46ms/step - loss: 7.9274e-05


In [7]:
# Tahminleri yapma
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)


18/18 [==============================] - 0s 15ms/step


In [8]:
# Tahminleri ters çevirme (normalize edilmemiş haline geri getirme)
train_predict = scaler.inverse_transform(train_predict)
y_train = scaler.inverse_transform([y_train])
test_predict = scaler.inverse_transform(test_predict)
y_test = scaler.inverse_transform([y_test])


In [9]:
# Eğitim ve test hatası hesaplama
train_score = np.sqrt(mean_squared_error(y_train[0], train_predict[:, 0]))
test_score = np.sqrt(mean_squared_error(y_test[0], test_predict[:, 0]))
print(f"Eğitim hatası (RMSE): {train_score:.2f}")
print(f"Test hatası (RMSE): {test_score:.2f}")

Eğitim hatası (RMSE): 1.49
Test hatası (RMSE): 7.19


In [10]:
# Tahminleri görselleştirme
train_plot = np.empty_like(close_prices_scaled)
train_plot[:,:] = np.nan
train_plot[look_back:len(train_predict)+look_back, :] = train_predict

test_plot = np.empty_like(close_prices_scaled)
test_plot[:,:] = np.nan
test_plot[len(train_predict)+(look_back*2)+1:len(close_prices_scaled)-1, :] = test_predict

plt.figure(figsize=(12,6))
plt.plot(scaler.inverse_transform(close_prices_scaled), label="Gerçek Fiyatlar")
plt.plot(train_plot, label="Eğitim Tahminleri", alpha=0.8)
plt.plot(test_plot, label="Test Tahminleri", alpha=0.8)
plt.legend()
plt.show()

ValueError: could not broadcast input array from shape (545,1) into shape (543,1)